<a href="https://colab.research.google.com/github/LambdaTheda/DS-Unit-2-Kaggle-Challenge/blob/master/PT5_u2s2m3_Cros_Val_223_ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [11]:
!pip install category_encoders==2.*

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

### Exploratory Data Analyses

In [13]:
train.shape, test.shape

((59400, 41), (14358, 40))

In [14]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


### Data Pre-Processing/Wrangling

In [0]:
#Define a function to wrangle Train, Validate, adn Test sets

import numpy as np

def wrangle(X):
  # Wrangle train, validate, and test sets in the same way

  X = X.copy()

  '''
  About 3% of the time, latitude has small values near zero,
  outside Tanzania, so we'll treat these values like zero.
  '''
  X['latitude'] = X['longitude'].replace(-2e-08, 0)

  '''
  When columns have zeros and shouldn't, they are like null values.
  So we will replace the zeros with nulls, and impute missing values later.
  '''
  cols_with_zeros = ['longitude', 'latitude', 'construction_year,
                     'gps_height', 'population']
                     
  '''
  Also create a "missing indicator" column, because the fact that
  values are missing may be a predictive signal. (Trying to prevent data leakage?)
  '''
  for col in cols_with_zeros:
      X[col] = x[col].replace(0, np.nan)
      X[col+'MISSING'] = X[col].isnull()

  # Drop duplicate colums
  duplicates = ['quantity_group', 'payment_type']
  X = X.drop(columns = duplicates) 

  # Drop 'recorded_by' (never varies) and 'id' (always varies; random) columns
  unusable_variance = ['recorded_by', 'id']
  X = X.drop(columns = unusable_variance)   

  #Convert 'date_recorded' to datetime
  X['date_recorded'] = pd.to_datetime(x['date_recorded'], infer_datetime_format = True)

  #Extract components from 'date_recorded', then DROP the ORIGINAL column
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['date_recorded'] = X['date_recorded'].dt.day

  X = X.drop(columns = 'date_recorded')

  # Engineer feature: Water pump age = date_recorded - construction_year
  X['waterpump_age'] = X['year_recorded'] - X['construction_year']
 
  # Also create a "missing indicator" column
  X['years_MISSING'] = X['waterpump_age'].isnull()

  # Return the Wrangled Data Frame/features matrix
  return X
  
train = wrangle(train)
test = wrangle(test)

### Arrange data into X feature matrix and y target vector

In [0]:
# The 'status_group' column is the target
target = 'status_group'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns = [target]) # syntax?

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include = 'number').columns.tolist()

# Get a series with the cardinality of the non-numeric features
cardinality = train_features.select_dtypes(exclude = 'number').nunique()

# Get a list of all categorical features iwth cardinality <= 50
categorical_features = cardinality[cardinality<= 50].index.tolist() '''Pandas Index. tolist() function return a list of the values.                                                                     
                                                                       These are each a scalar type, which is a Python scalar (for str, int, float) 
                                                                       or a pandas scalar (for Timestamp/Timedelta/Interval/Period). ''' 
 # Combine the numeric and filtered categorical lists
 features = numeric_features + categorical_features

In [0]:
''' Arrange data into X feature matrix and y target vector so we 
    use all features, INCLUDING the high-cardinaltiy categoricals''' # Why do we want these now?

X_train = train.drop(columns = target)
y_train = train[target]
X_test = test

In [0]:
# Check shape of Train, test datasets
print(X_train.shape, X_test.shape)

**PREDICT TARGET VECTOR:**

Using RandomForestClassifier model and its hyperparameters

Measure its performance with a classification metric such as scoring='accuracy'

Use a categorical encoding that works for multi-class classification, such as OrdinalEncoder


In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform

In [0]:
# Instantiate classes
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(randome_state = 42)
)

# Set hyper? parameters with a dictionary
param_distributions = { ''' From https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html: param_distributionsdict or list of dicts
                            Dictionary with parameters names (string) as keys and distributions or lists of parameters to try. '''
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max-depth': [5, 10. 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0,1),
}

# Use scikit-learn for hyperparameter optimization with RandomizedSearchCV
search = RandomizedSearchCV(
    pipeline,
    param_distributions = param_distributions,
    n_iter = 10,
    cv = 3,
    scoring = 'accuracy',
    verbose = 10,
    return_train_score = True,
    n_jobs = -1
  
)

search.fit(X_train, y_train);

In [0]:
print('Best hyperparameter', search.best_params_)
print('Percent accuracy', search.best_score_)

In [0]:
# Make predictions for the test
best = ssearch.fit(X_train, y_train)
y_pred = best.(X-test)

In [0]:
# Submit to Kaggle
submission = sample_submission.copy()
submission['status_group'] = y_pred # why y_pred assigned on right side?
submission.to_csv('submission_randomsearchCV_1.csv', index = False)

**K Folds Cross Validation Notes**

From: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

k-fold cross-validation procedure for estimating the skill of machine learning models.

Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample.

The procedure has a single parameter called k that refers to the number of groups that a given data sample is to be split into.

Cross-validation is primarily used in applied machine learning to estimate the skill of a machine learning model on unseen data. That is, to use a limited sample in order to estimate how the model is expected to perform in general when used to make predictions on data not used during the training of the model.

 it generally results in a less biased or less optimistic estimate of the model skill than other methods, such as a simple train/test split.

The general procedure is as follows:

1) Shuffle the dataset randomly.

2) Split the dataset into k groups

3) For each unique group:
   a) Take the group as a hold out or test data set

   b) Take the remaining groups as a training data set

   c) Fit a model on the training set and evaluate it on the test set

   d) Retain the evaluation score and discard the model

   e) Summarize the skill of the model using the sample of model evaluation 
      scores

results of a k-fold cross-validation run are often summarized with the mean of the model skill scores. It is also good practice to include a measure of the variance of the skill scores, such as the standard deviation or standard error.

important that any preparation of the data prior to fitting the model occur on the CV-assigned training dataset within the loop rather than on the broader data set. This also applies to any tuning of hyperparameters. A failure to perform these operations within the loop may result in data leakage and an optimistic estimate of the model skill.

A poorly chosen value for k may result in a mis-representative idea of the skill of the model, such as a score with a high variance (that may change a lot based on the data used to fit the model), or a high bias, (such as an overestimate of the skill of the model).

**Three common tactics for choosing a value for k are as follows:**

**Representative:** The value for k is chosen such that each train/test group of data samples is large enough to be statistically representative of the broader dataset.

**k=10:** The value for k is fixed to 10, a value that has been found through experimentation to generally result in a model skill estimate with low bias a modest variance.

**k=n:** The value for k is fixed to n, where n is the size of the dataset to give each test sample an opportunity to be used in the hold out dataset. This approach is called leave-one-out cross-validation.